# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

# Add the 05_src directory to the path so we can import local modules
%run ../../01_materials/labs/update_path.py

# Set up logging
from utils.logger import get_logger
_logs = get_logger(__name__)

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob


# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")    # load the env variable 
_logs.info(f'Loading price data from: {PRICE_DATA}')
parquet_files = glob(os.path.join(PRICE_DATA, "***/**/*.parquet"), recursive = True)  # load all the parquet files path into list
_logs.info(f'Found {len(parquet_files)} parquet files for reading back into Dask.')


2026-01-18 20:12:05,825, 2411419995.py, 7, INFO, Loading price data from: ../../05_src/data/prices/
2026-01-18 20:12:06,056, 2411419995.py, 9, INFO, Found 3102 parquet files for reading back into Dask.


In [4]:
parquet_files

['../../05_src/data/prices\\ABCB\\ABCB_1994\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1994\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1995\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1995\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1996\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1996\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1997\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1997\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1998\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1998\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1999\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1999\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2000\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2000\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2001\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2001\\part.1.parquet',
 '../../

In [5]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker") # read parquet files data into lazy dask dataframe
_logs.info(f'Successfully loaded data into Dask DataFrame.')

2026-01-18 20:12:06,544, 1115739733.py, 2, INFO, Successfully loaded data into Dask DataFrame.


In [6]:
print(dd_px)

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close Volume  source   Year
npartitions=89                                                                                    
ABCB            datetime64[ns]  float64  float64  float64  float64   float64  int64  string  int32
ACN                        ...      ...      ...      ...      ...       ...    ...     ...    ...
...                        ...      ...      ...      ...      ...       ...    ...     ...    ...
ZYXI                       ...      ...      ...      ...      ...       ...    ...     ...    ...
ZYXI                       ...      ...      ...      ...      ...       ...    ...     ...    ...
Dask Name: setindex, 2 expressions
Expr=SetIndex(frame=ReadParquetFSSpec(eee6358), _other='ticker', options={})


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.
import pandas as pd

# Add lag variables for Close and Adj Close variables for each ticker values

dd_shift = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1),            
							   Adj_Close_lag_1=x['Adj Close'].shift(1) ), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
					'Adj_Close_lag_1': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

In [8]:
# Add returns and hi_lo_range 

dd_feat = dd_shift.assign(
    returns = lambda x: (x['Close']/x['Close_lag_1']) - 1,
    hi_lo_range = lambda x: x['High']-x['Low']
)

In [9]:
dd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ABCB,1994-05-19,5.065844,5.369794,5.065844,5.268477,3.110148,134000.0,ABCB.csv,1994,NaN,NaN,NaN,0.303951
ABCB,1994-05-20,5.167161,5.369794,5.065844,5.268477,3.110148,54000.0,ABCB.csv,1994,10.85,10.224082,-0.514426,0.303951
ABCB,1994-05-23,5.268477,5.268477,5.065844,5.065844,2.990528,16200.0,ABCB.csv,1994,10.58,9.969661,-0.521187,0.202634
ABCB,1994-05-24,5.268477,5.268477,5.065844,5.065844,2.990528,7300.0,ABCB.csv,1994,11.07,10.431392,-0.542381,0.202634
ABCB,1994-05-25,5.167161,5.167161,5.065844,5.065844,2.990528,12900.0,ABCB.csv,1994,11.10,10.459661,-0.543618,0.101317


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [10]:
# Write your code below.

pandas_df_feat = dd_feat.compute()  # 

In [11]:
pandas_df_feat["returns_ma_10"] = (
    pandas_df_feat
    .groupby("ticker")["returns"]
    .rolling(10)
    .mean()
    .reset_index(level=0, drop=True)
)
_logs.info('Moving average calculation complete. New column: returns_ma_10')

2026-01-18 20:12:59,703, 1702960900.py, 8, INFO, Moving average calculation complete. New column: returns_ma_10


In [12]:
pandas_df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
ticker,,,,,,,,,,,,,,
ABCB,1994-05-19,5.065844,5.369794,5.065844,5.268477,3.110148,134000.0,ABCB.csv,1994,NaN,NaN,NaN,0.303951,NaN
ABCB,1994-05-20,5.167161,5.369794,5.065844,5.268477,3.110148,54000.0,ABCB.csv,1994,5.268477,3.110148,0.000000,0.303951,NaN
ABCB,1994-05-23,5.268477,5.268477,5.065844,5.065844,2.990528,16200.0,ABCB.csv,1994,5.268477,3.110148,-0.038462,0.202634,NaN
ABCB,1994-05-24,5.268477,5.268477,5.065844,5.065844,2.990528,7300.0,ABCB.csv,1994,5.065844,2.990528,0.000000,0.202634,NaN
ABCB,1994-05-25,5.167161,5.167161,5.065844,5.065844,2.990528,12900.0,ABCB.csv,1994,5.065844,2.990528,0.000000,0.101317,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYXI,2020-03-26,10.230000,11.430000,10.230000,11.100000,11.100000,189500.0,ZYXI.csv,2020,10.350000,10.350000,0.072464,1.200001,0.024000
ZYXI,2020-03-27,10.700000,10.980000,10.060000,10.300000,10.300000,145000.0,ZYXI.csv,2020,11.100000,11.100000,-0.072072,0.919999,0.007731
ZYXI,2020-03-30,10.160000,11.060000,10.160000,10.800000,10.800000,162300.0,ZYXI.csv,2020,10.300000,10.300000,0.048544,0.900001,0.018992


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)  
Yes, it was necessary to convert the data to Pandas in this case because we are working with a relatively small dataset. Although the same operations could be performed using Dask, it is not advisable to do so for small datasets because Dask introduces additional overhead related to task scheduling and parallel execution. For smaller data sizes, Pandas is simpler, faster, and more efficient, making it the more appropriate choice. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.